# Programmation de la classe noeud

In [23]:
import networkx as nx
import pandas as pd
import numpy as np

## Travail sur le graphe de conjonction

In [ ]:
def construct_conjonctive (schedule):
    pass

In [ ]:
def print_conjonctive(schedule) :
    pass

In [ ]:
def longest_path(schedule):
    '''
    output : python dictionnary
        - value : the length of the longest past
        - path : the tasks of N in the path.
    '''
    G = construct_conjonctive(schedule)
    res = {'value' : 0 , 'path' : []}


    # recherche du plus long chemin
    return(res)



## Structure de la classe Noeuds

- intégration du diagramme uml
- présentation des variables et des méthodes

In [18]:
class Node :

    def __init__ (self , N , a , d , q) :

        self._N = []
        self._a = {}
        self._d = {}
        self._q = {}

        k = 0

        for i in N :

            self._N.append(i)
            self._a[i] = a[k]
            self._q[i] = q[k]
            self._d[i] = d[k]

            k+=1

    def _to_string(self) :

        '''
        print of the node
        '''

        print("Task : " , end =" ")

        for i in self._N :
            print(i , end = " ")
        print()
        print("       ",end = " ")

        for i in self._N :
            s = str(self._a[i])
            print(s , end = " ")
        print()
        print("       ",end = " ")
        for i in self._N :
            s = str(self._d[i])
            print(s , end = " ")
        print()
        print("       ",end = " ")
        for i in self._N :
            s = str(self._q[i])
            print(s , end = " ")

    

    # 

    def _schrage_schedule(self) :
        pass

    
    def _schrage_makespan(self) :
        '''
        output : {'makespan' : int , 'longest_path : int []}

            'longest_path' : list of element of self._N.
            'makespan' : longueure du plus long chemin.
        '''

        res = {'makespan' : 0 , 'longest_path' : []}

        shr_schedule = self._schrage_schedule()

        long_path = longest_path(shr_schedule)

        res["longest_path"] = long_path

        return(res)



    def _critical_search(self) :

        '''
        output : python dictionnary
            - makespan : time of the schedule
            - 
        '''
        
        output_schrage = self._schrage_makespan()

        res = {'makespan' : output_schrage['makespan']} # dictionnary to return
        

        path = output_schrage['longest_path']

        # variables to return

        jp = path[-1] # last task of the schrage schedule
        target = self._q[jp]
        J = [jp]
        jc = -1
        status = ''

        for i in range(len(path)-2 , -1 , -1) :

            jk = path[i]
            buff = self._q[jk]

            if (buff<target) :
                jc = jk
                break
            else :
                J.append(jk)

        
        if(jc==-1):
            # critical task not found --> schrage schedule is optimal
            b = True
            J = []
            status = 'optimal'
        else : 
            status = 'not_optimal'

        
        # new values for the dictionnary
        res['J'] = J
        res['jc'] = jc
        res['status'] = status


        return(res)
        













Exemple de l'utilisation de la fonction **_to_string**

In [19]:
''' 
Node of the subject :
'''

N = list(range(1,8))
a = [10,13,11,20,30,0,30]
d = [5,6,7,4,3,6,2]
q = [7,26,24,21,8,17,0]

e = Node(N,a,d,q)
e._to_string()

Task :  1 2 3 4 5 6 7 
        10 13 11 20 30 0 30 
        5 6 7 4 3 6 2 
        7 26 24 21 8 17 0 